In [5]:
import pandas as pd

import BRADA as brada
print(brada.__dist_name__)
print('Version:',brada.__version__)

BRADA - BR Distribuidora Advanced Analytics Framework
Version: 0.0.1


In [6]:
pequim = pd.read_csv('datasets/BeijingPM20100101_20151231.csv')
pequim.dropna(inplace=True)

pequim['date'] = pequim['year'].astype(str) + '-' + pequim['month'].astype(str) + '-' + pequim['day'].astype(str)
pequim.drop_duplicates(subset='date', inplace=True)
pequim['date'] = pd.to_datetime(pequim['date'])
pequim.sort_values('date', inplace=True)
pequim.set_index('date', inplace=True)
pequim.index = pd.DatetimeIndex(pequim.index).to_period('D')

pequim.columns

Index(['No', 'year', 'month', 'day', 'hour', 'season', 'PM_Dongsi',
       'PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post', 'DEWP', 'HUMI',
       'PRES', 'TEMP', 'cbwd', 'Iws', 'precipitation', 'Iprec'],
      dtype='object')

In [7]:
black_friday = pd.read_csv('datasets/BlackFriday.csv')
black_friday.columns                           

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [8]:
fb = pd.read_csv('datasets/FB.csv')
fb.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [9]:
fb['Date'] = pd.to_datetime(fb['Date'])
fb = fb.set_index('Date')

In [10]:
fb.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-11-16,106.839996,107.870003,106.620003,107.320000,107.320000,22129600
2015-11-23,107.190002,107.470001,104.389999,105.449997,105.449997,64538400
2015-11-30,105.839996,107.919998,103.349998,106.180000,106.180000,112055400
2015-12-07,106.480003,106.910004,101.910004,102.120003,102.120003,103276000
2015-12-14,102.290001,107.750000,101.459999,104.040001,104.040001,126762800


In [ ]:
brada.automl()

In [ ]:
def continuous_to_categorical(df, continuos_columns=[], bins=5):
    
    import pandas as pd
    
    df_temp = df.copy()
    
    if len(continuos_columns) == 0:
        continuos_columns = df_temp.columns
    
    for col in continuos_columns:
        
        cat_col = pd.cut(df_temp[col], bins=bins)
        cat_col = cat_col.apply(lambda x: np.mean([x.left, x.right]))
        new_col_name = col + '_categorical'
        df_temp.loc[:, new_col_name] = cat_col
        
    return df_temp

In [ ]:
df_temp = continuous_to_categorical(fb, bins=10)

In [ ]:
df_temp['Close_categorical'].value_counts()

In [ ]:
(94*0.6*5.42)-600